In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
import networkx as nx
import warnings
warnings.filterwarnings('ignore')
# no warnings will be printed from now on


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Seting the directory 
%cd /content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle  

/content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle


### Loading data

In [ ]:
df_links = pd.read_csv("df_links_scaled.csv")[['id','ups', 'successors_name', 'predecessors_name','Ctr_degree_name', 'Ctr_cless_name', 'Ctr_Eigen_name', 'page_rk_name',
                                               'successors_prt_id', 'predecessors_prt_id']]
df_links.head() 

,id,ups,successors_name,predecessors_name,Ctr_degree_name,Ctr_cless_name,Ctr_Eigen_name,page_rk_name,successors_prt_id,predecessors_prt_id
0,cqug90j,3.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.193494
1,cqug90k,3.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,0.313289
2,cqug90z,5.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.192089
3,cqug91c,1.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.192440
4,cqug91e,101.0,0.0,0.132467,0.132467,0.146519,-0.061107,0.132467,-1.206389,-0.193143


In [ ]:
df_links.shape 

(4234970, 10)

In [ ]:
df_topic  = pd.read_csv("topic.csv")
df_topic

,id,weight_topic
0,cqug90j,-0.400798
1,cqug90k,-0.400798
2,cqug90z,-0.400798
3,cqug91c,-0.400798
4,cqug91e,2.477223
...,...,...
4234965,crrbelu,-0.400798
4234966,crrbelv,-0.400798
4234967,crrbemp,-0.400798
4234968,crrbenh,-0.400798


In [ ]:
df_sent_analysis = pd.read_csv("sentiment_analysis.csv", lineterminator='\n')[['id', 'polarity', 'subjectivity']]
df_sent_analysis

,id,polarity,subjectivity
0,cqug90j,0,0.000000
1,cqug90k,-1,1.000000
2,cqug90z,0,0.000000
3,cqug91c,0,0.000000
4,cqug91e,1,0.427083
...,...,...,...
4234965,crrbelu,0,0.000000
4234966,crrbelv,0,0.000000
4234967,crrbemp,-1,0.400000
4234968,crrbenh,0,0.000000


In [ ]:
df_author = pd.read_csv('df_author_scaled.csv')[['id', 'successors_auth', 'predecessors_auth', 'Ctr_dg_autor', 'Ctr_Eigen_auth', 'page_rk_auth', 'freq_occ_author']]
df_author

,id,successors_auth,predecessors_auth,Ctr_dg_autor,Ctr_Eigen_auth,page_rk_auth,freq_occ_author
0,cqug90j,-0.046253,-0.300397,-0.247372,-0.302522,-0.299207,-0.300596
1,cqug90k,-0.048731,-0.302489,-0.250612,-0.307353,-0.301532,-0.301107
2,cqug9uh,-0.048495,-0.302314,-0.250322,-0.306977,-0.301189,-0.300941
3,cquga7w,-0.034864,3.337207,2.361777,3.336656,3.337246,3.337239
4,cqugbfk,-0.048613,-0.302605,-0.250612,-0.307535,-0.301468,-0.301139
...,...,...,...,...,...,...,...
4234965,crrbedv,-0.048259,-0.301966,-0.249906,-0.305704,-0.300768,-0.300690
4234966,crrbego,-0.048613,-0.302256,-0.250363,-0.307296,-0.300846,-0.301034
4234967,crrbek0,-0.048554,-0.302198,-0.250280,-0.306705,-0.300990,-0.301014
4234968,crrbelf,-0.043007,-0.297840,-0.243258,-0.297440,-0.299695,-0.300159


In [ ]:
df = df_sent_analysis.merge(df_topic, on = 'id')
df = df.merge(df_links, on = 'id')
df = df.merge(df_author, on = 'id')
df 

,id,polarity,subjectivity,weight_topic,ups,successors_name,predecessors_name,Ctr_degree_name,Ctr_cless_name,Ctr_Eigen_name,page_rk_name,successors_prt_id,predecessors_prt_id,successors_auth,predecessors_auth,Ctr_dg_autor,Ctr_Eigen_auth,page_rk_auth,freq_occ_author
0,cqug90j,0,0.000000,-0.400798,3.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.193494,-0.046253,-0.300397,-0.247372,-0.302522,-0.299207,-0.300596
1,cqug90k,-1,1.000000,-0.400798,3.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,0.313289,-0.048731,-0.302489,-0.250612,-0.307353,-0.301532,-0.301107
2,cqug90z,0,0.000000,-0.400798,5.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.192089,-0.046961,-0.300165,-0.247704,-0.301609,-0.300020,-0.300565
3,cqug91c,0,0.000000,-0.400798,1.0,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.192440,-0.048731,-0.302663,-0.250737,-0.307536,-0.301554,-0.301107
4,cqug91e,1,0.427083,2.477223,101.0,0.0,0.132467,0.132467,0.146519,-0.061107,0.132467,-1.206389,-0.193143,-0.048672,-0.302605,-0.250654,-0.307497,-0.301551,-0.301139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,crrbelu,0,0.000000,-0.400798,NaN,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,0.331915,-0.048377,-0.302140,-0.250114,-0.307076,-0.301280,-0.301003
4234966,crrbelv,0,0.000000,-0.400798,NaN,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,0.331915,-0.048731,-0.302663,-0.250737,-0.307536,-0.301554,-0.301149
4234967,crrbemp,-1,0.400000,-0.400798,NaN,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,-0.190683,-0.048495,-0.302663,-0.250571,-0.307536,-0.301554,-0.301107
4234968,crrbenh,0,0.000000,-0.400798,NaN,0.0,-0.192790,-0.192790,-0.195690,-0.063330,-0.192790,-1.206389,0.431374,-0.048731,-0.302663,-0.250737,-0.307536,-0.301554,-0.301149


In [ ]:
# Transforming to log scale 
def trans_y(y):
  y=y-min_ups+1
  return y.apply(func=np.log)

def itrans(y_predict):
  y=y_predict.apply(func=np.exp)
  return y+min_ups-1

# Prediction

In [ ]:
train = df[df['ups'].notnull()]

test = df[df['ups'].isnull()]

In [ ]:
min_ups=np.min(train['ups'])
min_ups

-333.0

### Analysing the ups value

In [ ]:
df['ups'].describe()

count    3.218512e+06
mean     1.274048e+01
std      1.257486e+02
min     -3.330000e+02
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      6.761000e+03
Name: ups, dtype: float64

In [ ]:
y_train = trans_y(train['ups'])
y_train
# y_test=test['ups'].values

## Train and Test columns

In [ ]:
x_train = train.drop(["id" ,	"ups"], axis=1)
x_test = test.drop(["id" ,	"ups"] , axis=1) 

In [ ]:
x_test 

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
grad_boost =  GradientBoostingRegressor(loss = 'lad', subsample = 0.8, n_estimators = 350, learning_rate = 0.1, max_depth = 8) 

In [ ]:
%time
grad_boost.fit(x_train, y_train) 

In [ ]:
y_pred_grad_boost = grad_boost.predict(x_test) 

In [ ]:
test['predicted'] = y_pred_grad_boost
test['predicted'] = itrans(test['predicted'])
test['predicted'] = test['predicted'].astype(int)
## Creating a new dataframe which contains only needed columns
df_pred_gboost = test[['id', 'predicted']]
df_pred_gboost.reset_index(drop=True, inplace=True)

In [ ]:
df_pred_gboost.describe()

In [ ]:
df_pred_gboost.to_csv('dernierepreeed.csv', index= False)